In [1]:
import pandas as pd
import altair as alt
import json
import plotly.express as px
import geopandas as gpd

In [58]:
# 데이터 읽기
df = pd.read_csv('전체 순위.csv', encoding='utf-8')
df

,청소년 건강 종합 지수,Highest Score,Average,서울,부산,대구,인천,광주,대전,울산,세종,경기,강원,충북,충남,전북,전남,경북,경남,제주
0,최종 순위,-,-,15.00,2.00,5.00,9.00,14.00,6.00,3.00,4.00,10.00,12.00,1.00,7.00,17.00,16.00,8.00,11.00,13.00
1,총점,76.02,52.21,38.51,68.22,55.38,52.58,43.71,55.23,62.54,62.27,51.11,49.45,76.02,54.62,32.46,35.88,53.03,50.56,45.96
2,개인건강 점수,83.41,51.96,46.24,83.41,60.62,39.47,44.61,37.65,77.78,42.15,45.28,46.52,62.16,49.17,14.40,41.15,67.08,83.11,42.54
3,외부환경요인 점수,89.87,52.45,30.78,53.03,50.13,65.70,42.81,72.80,47.30,82.39,56.94,52.37,89.87,60.08,50.52,30.61,38.97,18.00,49.37


In [59]:
# 필요없는 열 제거
df.drop(columns=['Highest Score', 'Average'], inplace=True)
# 첫 번째 열을 인덱스로 설정
df.set_index('청소년 건강 종합 지수', inplace=True)
# 데이터 전치 (행과 열을 전환)
df_t = df.T

df_t

청소년 건강 종합 지수,최종 순위,총점,개인건강 점수,외부환경요인 점수
서울,15.0,38.51,46.24,30.78
부산,2.0,68.22,83.41,53.03
대구,5.0,55.38,60.62,50.13
인천,9.0,52.58,39.47,65.70
광주,14.0,43.71,44.61,42.81
대전,6.0,55.23,37.65,72.80
울산,3.0,62.54,77.78,47.30
세종,4.0,62.27,42.15,82.39
경기,10.0,51.11,45.28,56.94
강원,12.0,49.45,46.52,52.37


In [60]:
# 인덱스 제목 변경
df_t.index.name = '지역'
# 컬럼 순서 변경
df_t = df_t[['최종 순위', '총점', '개인건강 점수', '외부환경요인 점수']]
# 인덱스를 컬럼으로 이동
df_t.reset_index(inplace=True)

df_t.to_csv('./지역 별 건강점수.csv')

In [116]:
df = pd.read_csv('./지역 별 건강점수.csv', encoding='utf-8', index_col='지역')
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17 entries, 서울 to 제주
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   최종 순위      17 non-null     float64
 1   총점         17 non-null     float64
 2   개인건강 점수    17 non-null     float64
 3   외부환경요인 점수  17 non-null     float64
dtypes: float64(4)
memory usage: 680.0+ bytes


## 시도 단위 데이터 매치

### v.1 - json

In [95]:
# 파일 불러오기
with open('skorea_provinces_geo_simple.json', 'r', encoding='utf-8') as f:
    korea_map = json.load(f)

# 각 지역의 이름 변경
for feature in korea_map['features']:
    if feature['properties']['name'] == '제주특별자치도':
        feature['properties']['name'] = '제주'
    elif feature['properties']['name'] == '경상남도':
        feature['properties']['name'] = '경남'
    elif feature['properties']['name'] == '경상북도':
        feature['properties']['name'] = '경북'
    elif feature['properties']['name'] == '전라남도':
        feature['properties']['name'] = '전남'
    elif feature['properties']['name'] == '전라북도':
        feature['properties']['name'] = '전북'
    elif feature['properties']['name'] == '충청남도':
        feature['properties']['name'] = '충남'
    elif feature['properties']['name'] == '충청북도':
        feature['properties']['name'] = '충북'
    elif feature['properties']['name'] == '강원도':
        feature['properties']['name'] = '강원'
    elif feature['properties']['name'] == '경기도':
        feature['properties']['name'] = '경기'
    elif feature['properties']['name'] == '세종특별자치시':
        feature['properties']['name'] = '세종'
    elif feature['properties']['name'] == '울산광역시':
        feature['properties']['name'] = '울산'
    elif feature['properties']['name'] == '대전광역시':
        feature['properties']['name'] = '대전'
    elif feature['properties']['name'] == '광주광역시':
        feature['properties']['name'] = '광주'
    elif feature['properties']['name'] == '인천광역시':
        feature['properties']['name'] = '인천'
    elif feature['properties']['name'] == '대구광역시':
        feature['properties']['name'] = '대구'
    elif feature['properties']['name'] == '부산광역시':
        feature['properties']['name'] = '부산'
    elif feature['properties']['name'] == '서울특별시':
        feature['properties']['name'] = '서울'

# 변경된 데이터 저장
with open('skorea_provinces_geo_simple_modified.json', 'w', encoding='utf-8') as f:
    json.dump(korea_map, f, ensure_ascii=False, indent=4)

In [97]:
# JSON 파일 불러오기
with open('skorea_provinces_geo_simple_modified.json', 'r', encoding='utf-8') as f:
    korea_map = json.load(f)


# 각 지역의 이름과 ID 확인
for feature in korea_map['features']:
    name = feature['properties']['name']
    code = feature['properties']['code']
    print(f"Name: {name}, Code: {code}")
    

Name: 제주, Code: 39
Name: 경남, Code: 38
Name: 경북, Code: 37
Name: 전남, Code: 36
Name: 전북, Code: 35
Name: 충남, Code: 34
Name: 충북, Code: 33
Name: 강원, Code: 32
Name: 경기, Code: 31
Name: 세종, Code: 29
Name: 울산, Code: 26
Name: 대전, Code: 25
Name: 광주, Code: 24
Name: 인천, Code: 23
Name: 대구, Code: 22
Name: 부산, Code: 21
Name: 서울, Code: 11


### v.2 - geojson

In [135]:
with open('./korea_geojson2.geojson', encoding='UTF-8') as f:
    data = json.load(f)
    
# 각 지역의 이름 변경
for feature in data['features']:
    if feature['properties']['CTP_KOR_NM'] == '제주특별자치도':
        feature['properties']['CTP_KOR_NM'] = '제주'
    elif feature['properties']['CTP_KOR_NM'] == '경상남도':
        feature['properties']['CTP_KOR_NM'] = '경남'
    elif feature['properties']['CTP_KOR_NM'] == '경상북도':
        feature['properties']['CTP_KOR_NM'] = '경북'
    elif feature['properties']['CTP_KOR_NM'] == '전라남도':
        feature['properties']['CTP_KOR_NM'] = '전남'
    elif feature['properties']['CTP_KOR_NM'] == '전라북도':
        feature['properties']['CTP_KOR_NM'] = '전북'
    elif feature['properties']['CTP_KOR_NM'] == '충청남도':
        feature['properties']['CTP_KOR_NM'] = '충남'
    elif feature['properties']['CTP_KOR_NM'] == '충청북도':
        feature['properties']['CTP_KOR_NM'] = '충북'
    elif feature['properties']['CTP_KOR_NM'] == '강원도':
        feature['properties']['CTP_KOR_NM'] = '강원'
    elif feature['properties']['CTP_KOR_NM'] == '경기도':
        feature['properties']['CTP_KOR_NM'] = '경기'
    elif feature['properties']['CTP_KOR_NM'] == '세종특별자치시':
        feature['properties']['CTP_KOR_NM'] = '세종'
    elif feature['properties']['CTP_KOR_NM'] == '울산광역시':
        feature['properties']['CTP_KOR_NM'] = '울산'
    elif feature['properties']['CTP_KOR_NM'] == '대전광역시':
        feature['properties']['CTP_KOR_NM'] = '대전'
    elif feature['properties']['CTP_KOR_NM'] == '광주광역시':
        feature['properties']['CTP_KOR_NM'] = '광주'
    elif feature['properties']['CTP_KOR_NM'] == '인천광역시':
        feature['properties']['CTP_KOR_NM'] = '인천'
    elif feature['properties']['CTP_KOR_NM'] == '대구광역시':
        feature['properties']['CTP_KOR_NM'] = '대구'
    elif feature['properties']['CTP_KOR_NM'] == '부산광역시':
        feature['properties']['CTP_KOR_NM'] = '부산'
    elif feature['properties']['CTP_KOR_NM'] == '서울특별시':
        feature['properties']['CTP_KOR_NM'] = '서울'

for x in data['features']:
    x['id'] = x['properties']['CTP_KOR_NM'] 

# 각 지역의 이름과 ID 확인
for feature in data['features']:
    name = feature['properties']['CTP_KOR_NM']
    code = feature['properties']['CTPRVN_CD']
    print(f"Name: {name}, Code: {code}")

    지역  코드
0   강원  42
1   경기  41
2   경남  48
3   경북  47
4   광주  29
5   대구  27
6   대전  30
7   부산  26
8   서울  11
9   세종  36
10  울산  31
11  인천  28
12  전남  46
13  전북  45
14  제주  50
15  충남  44
16  충북  43


In [136]:
df

,지역,최종 순위,총점,개인건강 점수,외부환경요인 점수
0,서울,15.0,38.51,46.24,30.78
1,부산,2.0,68.22,83.41,53.03
2,대구,5.0,55.38,60.62,50.13
3,인천,9.0,52.58,39.47,65.70
4,광주,14.0,43.71,44.61,42.81
5,대전,6.0,55.23,37.65,72.80
6,울산,3.0,62.54,77.78,47.30
7,세종,4.0,62.27,42.15,82.39
8,경기,10.0,51.11,45.28,56.94
9,강원,12.0,49.45,46.52,52.37


## 지역 코드 매칭

In [98]:
# 각 지역명에 해당하는 코드를 매핑하는 딕셔너리 생성
region_to_code = {
    "서울": 11,
    "경기": 31,
    "강원": 32,
    "충북": 33,
    "충남": 34,
    "전북": 35,
    "전남": 36,
    "경북": 37,
    "경남": 38,
    "제주": 39,
    "인천": 23,
    "대전": 25,
    "광주": 24,
    "대구": 22,
    "부산": 21,
    "울산": 26,
    "세종": 29
}

# '지역' 열을 기반으로 코드 열 추가
df['코드'] = df.index.map(region_to_code)

# 결과 확인
print(df.head())

    최종 순위     총점  개인건강 점수  외부환경요인 점수  코드
지역                                      
서울   15.0  38.51    46.24      30.78  11
부산    2.0  68.22    83.41      53.03  21
대구    5.0  55.38    60.62      50.13  22
인천    9.0  52.58    39.47      65.70  23
광주   14.0  43.71    44.61      42.81  24


In [103]:
df.index

Index(['서울', '부산', '대구', '인천', '광주', '대전', '울산', '세종', '경기', '강원', '충북', '충남',
       '전북', '전남', '경북', '경남', '제주'],
      dtype='object', name='지역')

### geojson 기준 지역명만 매칭

In [159]:
import pandas as pd
import json
import plotly.express as px

# 데이터프레임 생성
df = pd.read_csv('./지역 별 건강점수.csv', encoding='utf-8')

# geojson 파일 불러오기
with open('./korea_geojson2.geojson', encoding='UTF-8') as f:
    data_geojson = json.load(f)

# 각 지역의 이름 변경
for feature in data_geojson['features']:
    if feature['properties']['CTP_KOR_NM'] == '제주특별자치도':
        feature['properties']['CTP_KOR_NM'] = '제주'
    elif feature['properties']['CTP_KOR_NM'] == '경상남도':
        feature['properties']['CTP_KOR_NM'] = '경남'
    elif feature['properties']['CTP_KOR_NM'] == '경상북도':
        feature['properties']['CTP_KOR_NM'] = '경북'
    elif feature['properties']['CTP_KOR_NM'] == '전라남도':
        feature['properties']['CTP_KOR_NM'] = '전남'
    elif feature['properties']['CTP_KOR_NM'] == '전라북도':
        feature['properties']['CTP_KOR_NM'] = '전북'
    elif feature['properties']['CTP_KOR_NM'] == '충청남도':
        feature['properties']['CTP_KOR_NM'] = '충남'
    elif feature['properties']['CTP_KOR_NM'] == '충청북도':
        feature['properties']['CTP_KOR_NM'] = '충북'
    elif feature['properties']['CTP_KOR_NM'] == '강원도':
        feature['properties']['CTP_KOR_NM'] = '강원'
    elif feature['properties']['CTP_KOR_NM'] == '경기도':
        feature['properties']['CTP_KOR_NM'] = '경기'
    elif feature['properties']['CTP_KOR_NM'] == '세종특별자치시':
        feature['properties']['CTP_KOR_NM'] = '세종'
    elif feature['properties']['CTP_KOR_NM'] == '울산광역시':
        feature['properties']['CTP_KOR_NM'] = '울산'
    elif feature['properties']['CTP_KOR_NM'] == '대전광역시':
        feature['properties']['CTP_KOR_NM'] = '대전'
    elif feature['properties']['CTP_KOR_NM'] == '광주광역시':
        feature['properties']['CTP_KOR_NM'] = '광주'
    elif feature['properties']['CTP_KOR_NM'] == '인천광역시':
        feature['properties']['CTP_KOR_NM'] = '인천'
    elif feature['properties']['CTP_KOR_NM'] == '대구광역시':
        feature['properties']['CTP_KOR_NM'] = '대구'
    elif feature['properties']['CTP_KOR_NM'] == '부산광역시':
        feature['properties']['CTP_KOR_NM'] = '부산'
    elif feature['properties']['CTP_KOR_NM'] == '서울특별시':
        feature['properties']['CTP_KOR_NM'] = '서울'

for x in data_geojson['features']:
    x['id'] = x['properties']['CTP_KOR_NM']

# Choropleth 생성
fig = px.choropleth_mapbox(
    df,
    geojson=data_geojson,
    locations='지역',
    color='총점',  # 정규화된 총점을 기준으로 색상을 지정합니다.
    color_continuous_scale=px.colors.sequential.Reds,  # 색상 스케일을 지정합니다.
    mapbox_style="carto-positron",
    zoom=5.5,
    center={"lat": 35.757981, "lon": 127.661132},
    opacity=0.6,
    labels={'총점': '청소년 건강지표 총점'},  # 레이블을 지정합니다.
)

fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})  # 레이아웃을 업데이트합니다.
fig.show()


In [150]:
df['geo_region']

0       서울특별시
1       부산광역시
2       대구광역시
3       인천광역시
4       광주광역시
5       대전광역시
6       울산광역시
7     세종특별자치시
8        None
9         강원도
10       None
11       None
12       전라북도
13       전라남도
14       경상북도
15       경상남도
16    제주특별자치도
Name: geo_region, dtype: object

## Choropleth

In [117]:
# Choropleth 생성
choropleth = alt.Chart(alt.Data(values=korea_map)).mark_geoshape(
    stroke='black',
    strokeWidth=0.5
).project(
    type='mercator'
).encode(
    alt.Color('총점:Q', scale=alt.Scale(scheme='blues')),
    tooltip=[
        alt.Tooltip('properties.name:N', title='Province'),
        alt.Tooltip('총점:Q', title='Score')
    ]
).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(df, '지역', ['총점', '코드'])
).properties(
    width=800,
    height=600
)

# Choropleth 출력
choropleth

alt.Chart(...)

In [119]:
# 인덱스를 열로 변환
df.reset_index(inplace=True)

# Choropleth 생성
choropleth = px.choropleth(df, geojson=korea_map, locations='지역', 
                           color='총점', 
                           color_continuous_scale='blues',
                           range_color=(0, max(df['총점'])),
                           labels={'총점':'총점'}
                          )
choropleth.update_geos(projection_type="mercator")  # Mercator 투영법 사용
choropleth.update_layout(
    template='plotly_dark',
    plot_bgcolor='rgba(0, 0, 0, 0)',
    paper_bgcolor='rgba(0, 0, 0, 0)',
    margin=dict(l=0, r=0, t=0, b=0),
    height=350
)